In [1]:

import os
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as Data
import torch.optim as optim
from MNN import *
import pyedflib
import numpy as np
import h5py
from torch.autograd import Variable
import math
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix as cm
import  matplotlib.pyplot as plt
import random
import copy
from sklearn.model_selection import KFold
import gc
from ResultCode import cm_plot

##迭代次数为50
EPOCH = 80
#batch size 为500
BATCH_SIZE = 500
##学习率为0.01
LEARNING_RATE = 0.01
##动量因子为0.9
MOMENTUM = 0.9
##mlp隐藏单元
mlp_hsize = 300
##rnn隐藏单元
rnn_hsize =300
row = 0
use_gpu = False
h5file = "D:/WorkSpace/Shell/datafeature/"
tensor_data_path ="D:/WorkSpace/Shell/data/h5data/"
files= os.listdir(h5file) #得到文件夹下的所有文件名称
files_len = len(files)
channels = ['ECG','EMG','EOG-R','EOG-L','O2-A1','C4-A1','F4-A1']
Max_acc = []

#保存  训练测试结果
def saveResult(testresult,loss_train,loss_test,acc,koldi,mnn):
    np.savetxt('./result_new20/test_matrix/testresult'+koldi+'.txt',testresult)
    np.savetxt('./result_new20/loss/loss_train'+koldi+'.txt',loss_train)
    np.savetxt('./result_new20/acc/acc_'+koldi+'.txt',acc)
    np.savetxt('./result_new20/loss/loss_test'+koldi+'.txt',loss_test)
    torch.save(mnn,'/net/net'+koldi+'.pkl')  #entire net

#训练MNN
def mnnTrain(train_x,train_y,loss_all,mnn,loss_func):
    train_y_shape = train_y.shape[0]
    train_y=train_y.view(train_y.shape[0]).type(torch.LongTensor)

    if(use_gpu):
        train_x,train_y = train_x.cuda(),train_y.cuda()
    # mnn output
    output = mnn(train_x.type(torch.FloatTensor))
    loss = loss_func(output, train_y)

    # cross entropy loss
    loss_all = loss_all + loss
    optimizer.zero_grad()           # clear gradients for this training step
    loss.backward()                 # backpropagation, compute gradients
    optimizer.step()                # apply gradients
#画图  损失  准确率
def plot_acc_loss(x1,x2,train_acc,loss_train,loss_test,koidi):

    plt.plot(x1, train_acc, 'b.-', label = 'Train accuracy')
    plt.title('accuracy vs epoches')
    plt.xlabel('epoches')
    plt.ylabel('accuracy')
    plt.legend(["Test accuracy","Train accuracy"])
    plt.savefig("./result_new20/picture/accuracy"+koidi+".jpg")
    plt.show()


    plt.subplot(2,1,1)
    plt.plot(x2, loss_train, 'r.-', label = u'loss_train')
    plt.subplot(2,1,1)
    plt.plot(x2, loss_test, 'b.-', label = u'loss_test')
    plt.title('loss vs epoches')
    plt.ylabel('loss')
    plt.savefig("./result_new20/picture/loss"+koidi+".jpg")
    plt.show()

def del_collect(del_var):
    del del_var
    gc.collect()



def Set_Dataset(h5file,train_index,type = 0):
    global eeg_data,labels
    flag=0
    ##训练集
    for filei in train_index:
        fileh5 = h5py.File(h5file+files[filei], 'r')
#    print(fileh5[channels[5]].value.shape)

    #短时傅里叶变换  n_fft=5s*采样率  窗口重叠为70%即窗口滑动hop_length=n_fft*30%
    ##返回值为一个tensor,其中第一个维度为输入数据的batch size，
    # 第二个维度为STFT应用的频数，
    # 第三个维度为帧总数，
    # 最后一个维度包含了返回的复数值中的实部和虚部部分
    #拼接文件的数据
    # if filei == 0:
    #     stft_data = torch.stft(torch.from_numpy(fileh5[channels[5]].value),n_fft=5*sample_rate,hop_length=int(5*sample_rate*0.25))
    #     labels = torch.from_numpy(fileh5["labels"].value)
    # else:
    #     stft_data = torch.cat((stft_data,torch.stft(torch.from_numpy(fileh5[channels[5]].value),n_fft=5*sample_rate,hop_length=int(5*sample_rate*0.25))),dim = 0)
    #     labels = torch.cat((labels,torch.from_numpy(fileh5["labels"].value)),dim = 0)
    #
        if flag == 0:
            eeg_data_channel1 = torch.from_numpy(fileh5["F4-A1"].value)
            eeg_data_channel2 = torch.from_numpy(fileh5["EOG-L"].value)
            eeg_data = torch.cat((eeg_data_channel1,eeg_data_channel2),dim = 1)
            labels = torch.from_numpy(fileh5["labels"].value)
            flag=1

        else:
            eeg_data_channel1 = torch.from_numpy(fileh5["F4-A1"].value)
            eeg_data_channel2 = torch.from_numpy(fileh5["EOG-L"].value)
            eeg_data_new = torch.cat((eeg_data_channel1,eeg_data_channel2),dim = 1)
            eeg_data = torch.cat((eeg_data,eeg_data_new),dim = 0)
            labels = torch.cat((labels,torch.from_numpy(fileh5["labels"].value)),dim = 0)

    if type == 1:
        torch_dataset = Data.TensorDataset(eeg_data,labels.squeeze())
        train_loader = Data.DataLoader(dataset=torch_dataset, batch_size=BATCH_SIZE, shuffle=True,num_workers=1)##num_workers是加载文件的核心数
        return train_loader
    else:
        return eeg_data,labels.squeeze()

##提取训练数据
Sam=np.array(range(files_len)) #
New_sam=KFold(n_splits=5)
foldi = 0

print("开始")

for train_index,test_index in New_sam.split(Sam):
    
    foldi = foldi + 1


    print("读取测试数据开始---------")

    test_eeg_data,test_labels=Set_Dataset(h5file,test_index,type=0)

    mnn = MNNs()
    optimizer = torch.optim.SGD(mnn.parameters(),lr=LEARNING_RATE)
    loss_func = nn.CrossEntropyLoss()
    if(use_gpu):
        mnn = mnn.cuda()
        loss_func = loss_func.cuda()

    #定义数组
    Loss_list_train = []
    Loss_list_test = []
    Accuracy_list = []
    Kappa = []
    Loss_list2 = []
    Accuracy_list2 = []

    #我这里迭代了80次，所以x的取值范围为(0，80)，然后再将每次相对应的准确率以及损失率附在x上
    x1 = range(0, EPOCH)
    x2 = range(0, EPOCH)
    y1 = Accuracy_list
    y2 = Loss_list_train
    y3 = Loss_list_test
    best_acc = 0

    print("开始训练------")
    #训练MNN网络

    max_accuracy = 0

    for epoch in range(EPOCH):
        loss_all = 0
        loss_test_all = 0
        acc_train_all = 0
        step_all = 0
        print("开始训练模型---------")
        train_part_len = len(train_index)//5
        for train_part in range(5):
            if train_part == 0:
                train_part_index = train_index[train_part_len*train_part:train_part_len*(train_part+1)]
            elif train_part == 4:
                train_part_index = train_index[train_part_len*train_part:len(train_index)]
            else:
                train_part_index = train_index[train_part_len*train_part:train_part_len*(train_part+1)]
            train_loader = Set_Dataset(h5file,train_part_index,type=1)
            for step, (train_x, train_y) in enumerate(train_loader):  #设定训练数据
                train_y_shape = train_y.shape[0]
                train_y=train_y.view(train_y.shape[0])

                if(use_gpu):
                    train_x,train_y = train_x.cuda(),train_y.cuda()
                # mnn output
                output = mnn(train_x.type(torch.FloatTensor))
                loss_train = loss_func(output, train_y.type(torch.LongTensor))
                train_pred_y = torch.max(F.softmax(output), 1)[1].numpy().squeeze()
                # cross entropy loss
                loss_all = loss_all + loss_train
                optimizer.zero_grad()           # clear gradients for this training step
                loss_train.backward()                 # backpropagation, compute gradients
                optimizer.step()                # apply gradients
            step_all = step + 1 + step_all
        print("训练模型完毕")
        #准确率
        train_accuracy = accuracy_score(train_y,train_pred_y)
        loss_all_avg = loss_all/(step_all)
        Loss_list_train.append(loss_all_avg)


        # #测试集
        test_output = mnn(test_eeg_data.type(torch.FloatTensor))
        loss_test = loss_func(test_output,test_labels.type(torch.LongTensor))
        Loss_list_test.append(loss_test)
        pred_y = torch.max(test_output, 1)[1].squeeze()

        mnn(test_eeg_data.type(torch.FloatTensor))

        # #混淆矩阵
        matrix = cm(test_labels.numpy().squeeze(),pred_y)
        # #准确率
        accuracy = accuracy_score(test_labels.numpy().squeeze(),pred_y)
        Accuracy_list.append(accuracy)

        #accuracy = sum(test_labels1 == pred_y)/ test_labels.numpy().squeeze().size
        print(str(foldi)+'_Epoch: ', epoch+1, '| train loss: %.4f' % loss_all_avg,'| train accuracy: %.4f' %train_accuracy,
          '| test loss: %.4f' % loss_test,'| test accuracy: %.4f' % accuracy)

        gc.collect()

    # #混淆矩阵
    matrix = cm(test_labels.numpy().squeeze(),pred_y.numpy().squeeze())


    # Max_acc.append(accuracy)

    print("结束训练网络-------")
    #保存
    result1 = np.array(matrix)
        #result4 = np.array(Loss_list_test)
    result2 = np.array(Loss_list_train)
    result3 = np.array(Loss_list_test)
    result4 = np.array(Accuracy_list)
    saveResult(result1,result2,result3,result4,"_"+str(foldi).zfill(2),mnn)
    #画图及保存
    plot_acc_loss(x1,x2,y1,y2,y3,"_"+str(foldi).zfill(2))
    Kappa.append(cm_plot(test_labels.numpy().squeeze(),pred_y,"_"+str(foldi)))
    result5 = np.array(Kappa)
    np.savetxt('./result_new20/kappa/kappa_'+str(foldi)+'.txt',result5)
    torch.cuda.empty_cache() 
print("结束")





OSError: [WinError 1455] 页面文件太小，无法完成操作。 Error loading "D:\Software\Anaconda\envs\pytorch_gpu\lib\site-packages\torch\lib\caffe2_detectron_ops_gpu.dll" or one of its dependencies.